In [37]:
import numpy as np
import pandas as pd
from siuba import *
import altair as alt
import altair_saver
from plotnine import *
from shared_utils import geography_utils
from shared_utils import altair_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [38]:
#import _clean_data
#df = _clean_data.make_clean_data()

In [39]:
import _dla_utils

In [40]:
df= pd.read_parquet("dla_df.parquet")

In [41]:
df.head()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,...,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaT,NaT,NON-MPO,5904,2018.0,Humboldt County
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,...,nan,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,32D0,2018.0,Mendocino County
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,...,nan,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,4820,2018.0,Humboldt County
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,...,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,5924,2018.0,Sacramento County
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,...,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,5924,2018.0,Sacramento County


## Testing Agg Functions

In [42]:
df1= _dla_utils.calculate_data_head(df, "total_requested", aggregate_by="mpo", aggfunc="mean")

In [43]:
df1

,mpo,total_requested
24,SLOCOG,1.786000e+06
18,SCAG,1.763596e+06
19,SDAG,1.306846e+06
8,KCOG,1.023312e+06
12,MTC,8.700922e+05
26,STANCOG,8.314157e+05
13,NON-MPO,7.535195e+05
14,SACOG,7.348009e+05
5,FCOG,6.495966e+05
30,TMPO,6.440739e+05


In [44]:
df2 = _dla_utils.count_all_years(df)

In [45]:
df3= _dla_utils.find_top(df)

In [46]:
df3.head(20)

,value,count,variable
0,CML,3293,prefix
1,HSIPL,3282,prefix
2,STPL,2779,prefix
3,ER,1590,prefix
4,BRLO,1402,prefix
5,BRLS,904,prefix
6,ATPL,587,prefix
7,RPSTPL,576,prefix
8,HPLUL,332,prefix
9,SRTSL,331,prefix


## Testing Chart Functions

In [47]:
_dla_utils.basic_bar_chart((df2>>filter(_.dist==4)), "prepared_y","unique_prefix")

alt.Chart(...)

In [48]:
_dla_utils.basic_bar_chart((df3>>filter(_.variable=="primary_agency_name")),
                           "count", "value")

alt.Chart(...)

In [49]:
def labeling(word):
    # Add specific use cases where it's not just first letter capitalized
    LABEL_DICT = { "prepared_y": "Year",
              "dist": "District",
              "total_requested": "Total Requested",
              "fed_requested":"Fed Requested",
              "ac_requested": "Advance Construction Requested",
              "nunique":"Number of Unique",
              "project_no": "Project Number"}
    
    if (word == "mpo") or (word == "rtpa"):
        word = word.upper()
    elif word in LABEL_DICT.keys():
        word = LABEL_DICT[word]
    else:
        word = word.replace('n_', 'Number of ').title()
        word = word.replace('unique_', "Number of Unique ").title()
        word = word.replace('_', ' ').title()
    
    return word

In [143]:
def basic_scatter_chart(df, col, aggregate_by, colorcol, chart_title_col, chart_title=[]):
    
    if chart_title != [""]:
        chart_title = (f"Highest {labeling(col)} by {labeling(aggregate_by)}")
        save= ""
        
    elif chart_title == [""]:
        chart_title = chart_title
        save = "_top_20"
         
        
    chart = (alt.Chart(df)
             .mark_circle(size=60)
             .encode(
                 x=alt.X(aggregate_by, title=labeling(aggregate_by)),
                 y=alt.Y(col, title=labeling(col)),
                 #column = "payment:N",
                 color = alt.Color(colorcol,
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_SEQUENTIAL_COLORS),
                                      legend=alt.Legend(title=(labeling(col)))
                                  ))
             .properties( 
                          title = (chart_title))
    )

    chart=styleguide.preset_chart_config(chart)
    # chart.save(f"./chart_outputs/scatter_{col}_by_{aggregate_by}{save}.png")
    
    return chart

In [144]:
basic_scatter_chart((df3>>filter(_.variable=="prefix")), "count", "value", "count", "variable", chart_title="top 20")

alt.Chart(...)

In [117]:
basic_scatter_chart(df2, "unique_primary_agency_name", "total_requested_sum", "dist")

alt.Chart(...)

In [14]:
_dla_utils.basic_line_chart((df3>>filter(_.variable=="prepared_y")), "value","count")

alt.Chart(...)